In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib notebook

# Introduction to Data Storage

##### Version 0.1

***

By AA Miller (Northwestern/CIERA)  
15 July 2022

[Session 15](https://github.com/LSSTC-DSFP/LSSTC-DSFP-Sessions/tree/main/Sessions/Session15) is primarily concerned with handling our data with efficiency.

Ideally, for any and every task we want to devire solutions that operate *faster*. 

This can be accomplished many different ways:

$~~~~~~$build algorithms that execute faster

$~~~~~~$spread calculations over many different computers simultaneously

$~~~~~~$find a compact storage solution for the data so it can be accessed more quickly

In our introduction to this session we will start with data storage, and discuss fast algorithms as a challenge problem. 

## Problem 1) IMDb Data

Throughout the session we will use information from the [Internet Movie Database (IMDb)](https://www.imdb.com/) to illustrate various principles regarding data storage.

For this notebook, we will use a [google sheets](https://docs.google.com/spreadsheets/d/1B-C7uJFrVNGpAXsGE6_xymfFVSKhwnIsI_RewkkmGa0/edit?usp=sharing) spreadsheet to examine this data (later in the week we will explore the same data stored in a database). 

A quick note on the provenance of this data. The files we have used to populate this data set are from [this website](https://relational.fit.cvut.cz/dataset/IMDb) and it may not be a list of every single movie on IMDb (there are no movies after 2004).

**Problem 1a**

Open the google sheet.$^\dagger$ What information is stored in the `movies` sheet of this file? 

How many movies are listed? 

$^\dagger$Note – the link points to a "view" of the data, you may find it useful to copy the file so that you have write access. 

*Write your answer here*

The `movies` sheet contains 4 columns: id, name, year, and rank. `id` appears to be a running index counting the movies, `name` is the title of the movie (and the file appears to be organized alphabetically by name), `year` is the release date for the film, and `rank` is the user score (10 = really good) for the film on IMDb. 

There are 388,269 movies in the file. This can (with an investment of time) be determined by scrolling all the way to the end of the file and seeing the top index. We can be more efficient and use the built-in count function to determine the number of movies `=COUNTA(B2:B)`.

**Problem 1b**

What information is stored in the `directors` sheet? 

How many directors are there? 

*Write your answer here*

The `directors` sheet includes 3 columns: id, First Name, Last Name. `id` is a running index, and the name columns are self-explanatory. 

There are 86,880 directors total in this sheet. 

**Problem 1c**

What information is stored in the `movies_directors` sheet?

How many rows are there? Does this make sense? Why?

*Write your answer here*

The `movies_directors` sheet includes 2 columns: `movie_id` and `director_id`. `movie_id` corresponds to the `id` column in the `movies` sheet, while `director_id` correspondss to the `id` in the directors sheet.

There are 371,180 total rows in this sheet. This answer does not really make sense – we know there are 388,269 movies in total, so I would expect just as many entries here. Perhaps there are a bunch of movies without directors? 

**Problem 1d**

Confirm your column identifications in **1c** by finding your favorite movie and making sure `movies_directors` correctly matches it with the proper director.

*Write your answer here*

My favorite movie is "Wayne's World" (at least it's top 5), and that has movie id = 360290. The director was Penelope Sphereis, who has director id = 75368. In the `movies_directors` sheet, this `movie_id` and `director_id` match up confirming my answer from **1c**. 

**Problem 1e**

What information is stored in the `movies_genres` sheet?

How many rows are there? Does this make sense? Why?

*Write your answer here*

The `movies_genres` sheet includes 2 columns: `movie_id` and `genre`. `movie_id` corresponds to the `id` column in the `movies` sheet, while `genre` is one of several potential genres for the movie.

There are 395119 total rows in this sheet. A quick glance at the sheet reveals two things: (i) there are several movies that do not have an identified genre (e.g., movie_id = 3), and (ii) there are movies that have multiple genres (e.g., movie_id = 8). 

It is hard to know how many rows to expect given these two facts, but given that many films fall into multiple genres it makes sense this sheet has more rows than there are total movies.

## Problem 2) Connections

We started this exercise with a goal of being efficient. And yet, the data have been organized across 4 different files (each sheet is effectively a unique csv file).  

**Problem 2a**

If we wanted to store all the data in a single sheet, how many rows would we need?

*write your answer here*

There are only 88,680 directors, so we could make a file that only has 88,680 rows and store all the information in the 4 sheets. 

**Problem 2b**

Supposing we did convert everything to a single sheet – how many columns would you need to store all the information in a single sheet? 

*Think about this for at least 30 sec, but you do not have to write down a full answer*

Below you will determine the numbers necessary to calculate the true answers to these problems, but we can get a sense of the order of magnitude in a relatively straightforward fashion. 

We need two columns for first and last name. We need $3N_\mathrm{movie}$ columns where $N_\mathrm{movie}$ is the most movies directed by *any* director. In a single sheet this would be `movie1`, `movie2`, ..., `movieN`, plus the associated `year` and `rank`. From `movies_directors` we see that director 8 has 35 movie credits, so this is *at least* 105 additional columns.

At maximum another $21N_\mathrm{movie}$ columns are required to account for the different genres for each movie. This is likely overly conservative, no film belongs to every genre – let's say 5 is a reasonable number per movie (e.g., Action, Sci-Fi, Fantasy, Comedy, Short). Then this is another 175 columns.

**Problem 2c**

Which has fewer total entries (the combination of a column and a row makes 1 entry) – all the data in one sheet, or the data spread across 4 sheets? 

*write your answer here*

For the four sheets there are: $4 \times 388{,}269 + 3 \times 88{,}680 + 2 \times 371{,}180 + 2 \times 395{,}119 = 3{,}351{,}714$ entries

For a single sheet there are: $282 \times 88{,}680 = 25{,}007{,}760$ entries

What we see here is that it is very inefficient/expensive to record "nothing". 

Suppose there is a director that only has one film credit. In the single sheet solution, we need to record that `movie2` = `NULL`, and `movie3` and so on. This information has a cost (storage), AND, there is essentially nothing learned from said information. 

The reason the data have been organized into several sheets (or shall we call them "data tables"; yeah! let's call them "tables") is that it allows us to store far less information. There are some columns that we otherwise would not need (in a single table there is no need for an `id`, whereas the multi-table solution has at least one id in every table). 

The `id` however is very powerful, as it is what allows the connection between the different tables.

**Problem 2d**

What is the release year for every movie directed by Martin Scorcese? 

*Hint* – do not spend more than 10 min working on this problem (you probably should not even spend a full 10 min). 

*write your answer here*

Scorsese has director `id` = 71645. 

If you are scrolling or use built in functions, you can find that the 40 films that Scorsese has directed are: [7842, 8183, 10702, 13395, 13804, 25192, 27108, 37304, 42328, 45312, 47130, 54209, 56304, 67388, 82967, 123849, 131780, 163603, 163730, 177369, 181766, 183593, 185704, 185751, 193231, 199510, 209158, 212717, 214872, 230947, 230963, 253470, 270971, 316709, 326155, 352863, 362473, 364368, 379931, 382309]

If we copy that list of movie numbers to the `movies` sheet, we can then use a `FILTER` to find the years:  
`=FILTER(C2:C1000000, A2:A1000000 = G10)`  where G10 is 7842. This filter can then be copied to all rows below the first to result in the following years (this is sorted by title alphabetically):

[1985, 1993, 1974, 1992, 1978, 2004, 1987, 1967, 2005, 1972, 1999, 1991, 1995, 1986, 2006, 2002, 1990, 1964, 1974, 1983, 1997, 2004, 1988, 1978, 1988, 1990, 1973, 1995, 1999, 1989, 1977, 1995, 1980, 1970, 1976, 1959, 1963, 1967, 1985, 2003]

## Problem 3) Leverage

Now that we know why the data has been organized in this way, we can leverage this unique structure in order to learn interesting properties of the data. 

**Problem 3a**

As a follow-up to a previous problem, according the the IMDb data, which director has directed the most movies?

*Hint* – there are probably several ways to figure this out, but I'll note that google sheets provides the ability to calculate aggregated statistics *after* the data have been grouped in a single category. This type of operation exists on many different platforms, and is typically called a `group by`. 

For google sheets, a `group by` is performed inside the built in `QUERY` function. For example, if you were to type the following into a cell:

`=QUERY(A1:D25, "select B, AVG(D) group by B", 1)`

this would look at all the data in columns A--D, and between rows 1--25, it would then select values in B and "group the data" (i.e., suppose this column listed city names and column D listed temperatures, then every entry for the same city would be "grouped"), then aggregate statistics are caluclated on D (in this example, the average temperature for the city would be calculated). The `1` at the end indicates a single header row to those being searched.

The results would be a column saying city and a column saying temperature, and then the average temp in each city would be recorded. 

*Hint 2* – run your query on only a few rows first to make sure it works in the expected fashion.

To determine who has directed the most movies, we can run the following command on the `movies_directors` table.

`=QUERY(A1:B400000, "select A, count(B) group by A order by count(B) desc limit 5", 1)`

*Note* – I have added an `order by count(B) desc` so that the results show the largest numbers first. I have also added a `limit 5` so that only the top results are shown (we don't care about all the people that have only directed one film at this time). 

From this we see that director `25116` has directed 616 movies (!!!)

|  director_id  |  count movie_id  |
| :---:    | :---:       |
|  25116        |  616  |
|56530	|554|
|30570	|530|
|1958	|360|
|24576	|345|

We can now scroll in the directors table to see that `director_id = 25116` corresponds to [Dave Fleischer](https://www.imdb.com/name/nm0281487/?ref_=fn_al_nm_1#director).

P.S. How in the world did Dave Fleischer have enough time to direct 616 movies in a single life time? 

P.P.S. If we need 616 columns for the single table/sheet solution, that is another factor of $\sim$20 more storage. Hopefully it's clear this would be a bad idea.

We now know who (according to IMDb) has directed the most movies ever.

You should feel a little unsatisfied with this solution, however. While we had a clever way to count the movies by director, we still had to click over to another sheet and scroll (or `cntl+f`) to find the name of the director.

It would be far more satisfying to use a single command that returns director name *and* number of movies. 

To do this in a single command requires a `JOIN` (i.e., data from two different tables/sheets must be combined). 

Sadly, joins are not possible in google sheets$^\dagger$ (or at least I've spent a lot of time trying, and I have not found anything close to a useful solution even for the moderate amount of data included in this project).

$^\dagger$If you know how to do this let me know.

To understand why the join operation is so powerful, consider the following question:

Which drama directed by Martin Scorsese has the highest IMDb rating? 

This question is not that complex, but, we need to do the following to solve it with our spreadsheet: 
  1. look up the Scorsese `director_id`
  2. use `movies_directors` to look up the movies Scorsese has directed
  3. use `movies_genres` to figure out which of those movies are dramas
  4. select and sort the subset of Scorsese drama `movies` based on score

That's a lot of clicking around and running multiple operations. 

We will see several other methods later this week that are capable of `JOIN`ing tables so that this can be accomplished quickly.

**Problem 3b**

In which year were the most movies made according to IMDb?

*write your answer here*

We can again use `group by` to solve this problem in the `movies` table:

`=QUERY(A1:C500000, "select C, count(C) group by C order by count(C) desc limit 25", 1)`

From this we find that 2002 had the most movies, with 12056.

## Challenge Problem) Algorithmic Speed

For the challenge problem we will consider a different type of efficiency – algorithms.

We will demonstrate the types of improvements that are possible even for simple algorithms, like "sorting".

Both `python` and `numpy` have build-in functions that can sort a list or array with a great deal of efficiency. In practice, I would never recommend writing your own function to this end, but we will demonstrate how much time can be saved via "clever" implementations of straightforward ideas. 

**Challenge a**

Write a "pure" python function to sort the numbers in a list. Restrict yourself to simple list operations (i.e., do not use `np.sort()`. 

*Hint* – we can sort a list by creating a new list, `sort_list`, and looping over all elements in the original list and inserting each element in the position where it is greater than the previous value and less than or equal to the next value in `sorted`.

In [48]:
def sortlist(unsort):
    '''Sort the unsorted list unsort
    
    Inputs
    ------
    unsort : list of numbers
    
    Returns
    -------
    sort_list : values of unsort after being sorted
    '''
    
    sort_list = []
    sort_list.append(unsort[0])
    if unsort[1] > unsort[0]:
        sort_list.append(unsort[1])
    else:
        sort_list.insert(0,unsort[1])
    for u in unsort[2:]:
        for idx, sl in enumerate(sort_list):
            n = len(sort_list)
            if u <= sl:
                sort_list.insert(idx, u)
                break
            elif idx + 1 == n:
                sort_list.insert(idx+1, u)
                break
    return sort_list

**Challenge b**

Confirm your algorithm works by running it on the following list: [4, 1, 6, 2, 4, 7, 9, 3, 8]

In [49]:
sortlist([4, 1, 6, 2, 4, 7, 9, 3, 8])

[1, 2, 3, 4, 4, 6, 7, 8, 9]

**Challenge c**

Create a list with 10000 random numbers. Time how long it takes to sort that list with your algorithm.

In [91]:
import time

np.random.seed(1662)

haha = list(np.random.randn(10000))

tstart = time.time()
sl = sortlist(haha)
tend = time.time()

print('It takes {:.4f} s to sort this list'.format(tend-tstart))

It takes 3.4232 s to sort this list


10000 elements is not that many, and 3 s is a lot of time. In this case the speed of the algorithm is quite slow because we have a nested for loop. As the sorted array becomes larger, we have to again and again consider all the same numbers while looking for the "right" place to insert our value. 

We can achieve dramatic improvements with a "divide and conquer" strategy. In particular, we will use the [`quicksort`](https://en.wikipedia.org/wiki/Quicksort) algorithm to avoid looking at the same numbers again and again. 

In brief, `quicksort` does the following: pick a single element of an array/list (called the "pivot"), move all values less than the pivot to its right, move all values greater than the pivot to its left.

The pivot is now in its correct position within the array. We can now perform the same operation on the left and on the right to recursively move everything into the correct order. Unlike the "simple" method, we do not need to "look at" the same values again and again and again.

**Challenge d**

Write a recursive `quicksort` algorithm to sort values in a list/array. 

*Hint* – use the last element of the array as the "pivot" each time you separate the data into high and low values.

In [86]:
def sort_pivot(l, r, arr):
    pivot, ptr = arr[r], l
    for i in range(l, r):
        if arr[i] <= pivot:
            arr[i], arr[ptr] = arr[ptr], arr[i]
            ptr += 1
    arr[ptr], arr[r] = arr[r], arr[ptr]
    return ptr

def quicksort(l, r, arr):
    if len(arr) == 1:
        return arr
    if l < r:
        pi = sort_pivot(l, r, arr)
        quicksort(l, pi-1, arr)
        quicksort(pi+1, r, arr)
    return arr

**Challenge e**

Confirm your algorithm works by sorting the short list [4, 1, 6, 2, 4, 7, 9, 3, 8].

In [87]:
short_list = [4, 1, 6, 2, 4, 7, 9, 3, 8]
quicksort(0, len(short_list)-1, short_list)

[1, 2, 3, 4, 4, 6, 7, 8, 9]

**Challenge f**

Time how long it take to sort the random number array using `quicksort`.

In [92]:
meh = haha.copy()
tstart = time.time()
sl = quicksort(0, len(haha)-1, meh)
tend = time.time()

print('It takes {:.4f} s to sort this list'.format(tend-tstart))

It takes 0.0388 s to sort this list


A factor of 100 improvement! 

**Challenge g**

Time how long it takes to sort the random number array using `np.sort()`. 

In [93]:
meh = haha.copy()

tstart = time.time()
sl = np.sort(meh)
tend = time.time()

print('It takes {:.4f} s to sort this list'.format(tend-tstart))

It takes 0.0068 s to sort this list


While your solution was quite clever, under the hood `numpy` is still another factor of 5 faster! 

The upside from this exercise – every time you find yourself creating a for loop, ask the question do I really need a for loop. For the most part you will always be working with software that is already optimized but the for loops that you write may not be...